In [1]:
!pip install keras-cv

In [16]:
import os
from tensorflow import keras
import losses, train, GhostFaceNets
import tensorflow as tf
import keras_cv

ModuleNotFoundError: No module named 'tensorflow.keras'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

[]


In [23]:
# Remove the below for better accuracies and keep it for faster training
keras.mixed_precision.set_global_policy("mixed_float16")

GhostFaceNetV1

GhostFaceNetV2

In [24]:
# ms1m-retinaface-t1 (MS1MV3) dataset
data_basic_path = 'drive/MyDrive/datasets/ms1m-retinaface-t1'
data_path = data_basic_path + '_112x112_folders'
eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'drive/MyDrive/datasets/faces_emore_112x112_folders'
eval_paths = ['drive/MyDrive/datasets/faces_emore/lfw.bin', 'drive/MyDrive/datasets/faces_emore/cfp_fp.bin', 'drive/MyDrive/datasets/faces_emore/agedb_30.bin']

In [8]:
#GhostFaceNetV2
# Strides of 2
basic_model = build_ghostnetv2(dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)


# Strides of 1
basic_model = build_ghostnetv2(dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [26]:
#Strides of 2
tt = train.Train(data_path,
    save_path='ghostnetv2_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path,
    save_path='ghostnetv2_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

>>>> L2 regularizer value from basic_model: 0.00025
>>>> L2 regularizer value from basic_model: 0.00025


In [27]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

>>>> Init type by loss function name...
>>>> Train arcface...
>>>> Init softmax dataset...
>>>> Image length: 760398, Image class length: 760398, classes: 10038
>>>> Use specified optimizer: <keras.optimizers.sgd.SGD object at 0x7ba504251950>
>>>> Add L2 regularizer to model output layer, output_weight_decay = 0.000500
>>>> Add arcface layer, arc_kwargs={'loss_top_k': 1, 'append_norm': False, 'partial_fc_split': 0, 'name': 'arcface'}, vpl_kwargs={'vpl_lambda': 0.15, 'start_iters': -5940, 'allowed_delta': 200}...
>>>> loss_weights: {'arcface': 1}

Learning rate for iter 1 is 0.10000000149011612, global_iterNum is 0


KeyboardInterrupt: 

CosFaceLoss

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.CosFaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.CosFaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

Subcenter ArcFace Loss

In [ ]:
# ms1m-retinaface-t1 (MS1MV3) dataset
data_basic_path = 'datasets/ms1m-retinaface-t1'
data_path = data_basic_path + '_112x112_folders'
eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'datasets/faces_emore_112x112_folders'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [13]:
# GhostFaceNetV2
# Strides of 2
basic_model = build_ghostnetv2(dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)


# Strides of 1
basic_model = build_ghostnetv2(dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)

ValueError: Input 0 of layer "conv2d_341" is incompatible with the layer: expected min_ndim=4, found ndim=2. Full shape received: (None, 960)

In [12]:
#GhostFaceNetV2
#Strides of 2
tt = train.Train(data_path,
    save_path='ghostnetv2_w1.3_s2_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path,
    save_path='ghostnetv2_w1.3_s1_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

>>>> L2 regularizer value from basic_model: 0.00025
>>>> L2 regularizer value from basic_model: 0.00025


In [ ]:
""" First, Train with `lossTopK = 3` """
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer, "lossTopK": 3},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50, "lossTopK": 3},
]
tt.train(sch, 0)

In [ ]:
""" Then drop non-dominant subcenters and high-confident noisy data, which is `>75 degrees` """
import data_drop_top_k
# data_drop_top_k.data_drop_top_k('./checkpoints/TT_mobilenet_topk_bs256.h5', '/datasets/faces_casia_112x112_folders/', limit=20)
new_data_path = data_drop_top_k.data_drop_top_k(tt.model, tt.data_path)

In [ ]:
""" Train with the new dataset again, this time `lossTopK = 1` """
tt.reset_dataset(new_data_path)

At this point you can continue training the model or start training again and considering the generated dataset as a new dataset

In [ ]:
# Continue training
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

In [ ]:
# Train the models from scatch using the new generated dataset

# New Cleaned Dataset
data_path = 'Path_To_New_Generated_Dataset'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [6]:
# GhostFaceNetV1
# Strides of 2
basic_model = build_ghostnetv1(dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)


# Strides of 1
basic_model = build_ghostnetv1(dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [7]:
# GhostFaceNetV2
# Strides of 2
basic_model = build_ghostnetv2(dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)


# Strides of 1
basic_model = build_ghostnetv2(dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [13]:
# GhostFaceNetV1
# Strides of 2
tt = train.Train(data_path,
    save_path='ghostnetv1_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

# Strides of 1
tt = train.Train(data_path,
    save_path='ghostnetv1_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

>>>> L2 regularizer value from basic_model: 0.00025
>>>> L2 regularizer value from basic_model: 0.00025


In [14]:
# GhostFaceNetV2
#Strides of 2
tt = train.Train(data_path,
    save_path='ghostnetv2_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path,
    save_path='ghostnetv2_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

>>>> L2 regularizer value from basic_model: 0.00025
>>>> L2 regularizer value from basic_model: 0.00025


In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

In [8]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print("GPU is available and detected by TensorFlow.")
    # Perform a simple operation to confirm usage
    a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
    b = tf.constant([[1.0, 1.0], [0.0, 1.0]])
    c = tf.matmul(a, b)
    print("Matrix multiplication result on GPU:")
    print(c)
else:
    print("No GPU available or detected by TensorFlow.")


No GPU available or detected by TensorFlow.


# Task
Change all the cells in the notebook to use `keras-cv` instead of `keras_cv_attention_models`.

## Install `keras-cv`

### Subtask:
Add a cell to install the `keras-cv` library.
